### Uso de claves de bucket de Amazon S3 con KMS para cifrar objetos

#### Problema
Necesita cifrar los objetos de S3 en reposo con una clave gestionada por el cliente (CMK) del Servicio de Administración de Claves (KMS) y asegurarse de que todos los objetos del bucket se cifran con la clave KMS de forma rentable.

#### Solución
Cree una clave gestionada por el cliente de KMS, configure su cubo de S3 para utilizar claves de cubo de S3 que hagan referencia a su CMK de KMS de AWS y configure una política de cubo de S3 que requiera el uso de KMS para todas las operaciones de S3:PutObject.
<br>
<br>
<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0310.png" width="550">


In [3]:
import boto3
import json
import random


region_aws = 'us-east-1'

kms = boto3.client('kms', region_name=region_aws)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
s3control = boto3.client('s3control', region_name=region_aws)
ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [2]:
# Create a bucket
bucket_name = 'cookbook-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket_name)
print("Bucket name: {}".format(bucket_name))

# Por defecto cuando creamos un bucket, las opciones de **Block all public access** (Bloquear todo el acceso público) 
# están desactivadas por defecto.

response = s3_client.put_public_access_block(
        Bucket=bucket_name,
        PublicAccessBlockConfiguration={
            'BlockPublicAcls': True,
            'IgnorePublicAcls': True,
            'BlockPublicPolicy': True,
            'RestrictPublicBuckets': True
        }
)

Bucket name: cookbook-141647


Crear una clave KMS para utilizarla en su bucket de S3 y almacene el ID de la clave en una variable de entorno:

In [5]:
response = kms.create_key(
    Description='AWSCookbook - This key is used to encrypt objects in S3',
    Tags=[
        {
            'TagKey': 'Name',
            'TagValue': 'AWSCookbook306Key'
        },
    ]
)
key_id = response['KeyMetadata']['KeyId']
print("Key ID: {}".format(key_id))

Key ID: b22ae65e-6ede-49b6-9eb7-c10eb646b4ba


In [6]:
# Create an alias to reference your key:
response = kms.create_alias(
    AliasName='alias/awscookbook306',
    TargetKeyId=key_id
)

Configure el bucket de S3 para utilizar una clave de bucket especificando su ID de clave KMS:

In [12]:
response = s3_client.put_bucket_encryption(
    Bucket=bucket_name,
    ServerSideEncryptionConfiguration={
        'Rules': [
            {
                'ApplyServerSideEncryptionByDefault': {
                    'SSEAlgorithm': 'aws:kms',
                    'KMSMasterKeyID': key_id
                },
                'BucketKeyEnabled': True
            },
        ]
    }
)


Cree una plantilla de política para el bucket para forzar el cifrado de todos los objetos:

In [13]:
policy_s3_encryption = {
  "Version": "2012-10-17",
  "Id": "PutObjectPolicy",
  "Statement": [
    {
      "Sid": "DenyUnEncryptedObjectUploads",
      "Effect": "Deny",
      "Principal": "*",
      "Action": "s3:PutObject",
      "Resource": "arn:aws:s3:::{}/*".format(bucket_name),
      "Condition": {
        "StringNotEquals": {
          "s3:x-amz-server-side-encryption": "aws:kms"
        }
      }
    }
  ]
}

In [15]:
# Apply the bucket policy to force encryption on all uploads:
response = s3_client.put_bucket_policy(
    Bucket=bucket_name,
    Policy=json.dumps(policy_s3_encryption)
)

#### Controles de validación

Suba un objeto al bucket de S3 con encriptación desde la línea de comandos. Verás que la subida se ha realizado con éxito:

In [19]:
print("En la terminal ejecute el siguiente comando:")
print("aws s3 cp new-report.png s3://{}/test-file.png --sse aws:kms --sse-kms-key-id {}".format(bucket_name, key_id))

En la terminal ejecute el siguiente comando:
aws s3 cp new-report.png s3://cookbook-141647/test-file.png --sse aws:kms --sse-kms-key-id b22ae65e-6ede-49b6-9eb7-c10eb646b4ba


Ahora, suba un objeto al bucket de S3 sin cifrar. Notarás que recibes un error en la línea de comandos. Esto indica que la política de bucket que configuraste está funcionando correctamente:

In [20]:
print("En la terminal ejecute el siguiente comando:")
print("aws s3 cp new-report.png s3://{}/test-file.png".format(bucket_name))

En la terminal ejecute el siguiente comando:
aws s3 cp new-report.png s3://cookbook-141647/test-file.png


#### Discusión
Al aplicar el cifrado a su bucket de S3, podría haber optado por utilizar una CMK administrada por AWS que Amazon S3 crea en su cuenta de AWS y administra por usted. Al igual que la CMK gestionada por el cliente, su CMK gestionada por AWS es única para su cuenta de AWS y su región. Solo Amazon S3 tiene permiso para utilizar esta CMK en su nombre. Puede crear, rotar y desactivar las CMK gestionadas por el cliente auditables desde la consola de AWS KMS. La documentación de S3 ofrece una explicación exhaustiva de las diferencias entre los tipos de cifrado admitidos en S3.

Cuando se encriptan los datos, estos están protegidos, pero hay que proteger la clave de encriptación. El [cifrado de sobre](https://docs.aws.amazon.com/es_es/kms/latest/developerguide/concepts.html#enveloping) es la práctica de cifrar los datos de texto plano con una clave de datos, y luego cifrar la clave de datos bajo otra clave.